<a href="https://colab.research.google.com/github/sally001020/DataVisualization/blob/main/cctv_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CCTV 데이터 시각화 


## 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import altair as alt

## 2. 데이터 적재

In [ ]:
# cctv 및 인구 데이터 적재 및 확인
df = pd.read_csv("https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv", engine='python')
df.sample(5)

,no,id,자치구,CCTV_규모_2015,CCTV_규모_2020,인구_규모_2015,인구_규모_2020
5,2,6,동대문구,1436,2315,373824,357014
0,6,1,종로구,1175,1729,163822,158996
24,12,25,강동구,725,2547,463321,463998
4,13,5,광진구,477,2556,375180,360109
18,23,19,영등포구,1278,3773,417811,407367


In [ ]:
# 지도 시각화를 위한 자치구 위치 데이터 적재 및 확인
offices = pd.read_csv("https://raw.githubusercontent.com/logistex/vd21/main/offices.csv", engine='python')
offices.sample(5)

,자치구,자치구청,위도,경도,주소
6,중랑구,중랑구청,37.606560,127.092624,대한민국 서울특별시 중랑구 신내동 662
21,서초구,서초구청,37.483574,127.032661,대한민국 서울특별시 서초구 서초2동 남부순환로 2584
4,광진구,광진구청,37.538533,127.082377,대한민국 서울특별시 광진구 자양1동 자양로 117
3,성동구,성동구청,37.563072,127.036669,대한민국 서울특별시 성동구 행당동 고산자로 270
17,금천구,금천구청,37.451853,126.902036,대한민국 서울특별시 금천구


In [ ]:
# 자치구 경계선에 관한 지도 데이터
seoul_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
seoul = alt.topo_feature(seoul_url, 'seoul_municipalities_geo')
seoul

UrlData({
  format: TopoDataFormat({
    feature: 'seoul_municipalities_geo',
    type: 'topojson'
  }),
  url: 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
})

## 3. 버블차트 시각화

### 3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의 


In [ ]:
# 회귀선 버블차트 시각화 함수 정의

def bubble(year, color):   
    bb = alt.Chart(df).mark_circle().transform_calculate(
        CCTV_비율 = 'datum.CCTV_규모_{0} / datum.인구_규모_{0} *100'.format(year)
    ).encode(
        x=alt.X('인구_규모_{0}'.format(year),
               scale = alt.Scale(domain=[100000,700000]),
                title = '인구 규모 (단위:명)'),
        y = alt.Y('CCTV_규모_{0}'.format(year),
                  scale = alt.Scale(domain=[0,7000]),
                 title = 'CCTV 설치 규모 (단위:대)'),
        color = alt.value(color),
        opacity = alt.value(0.5),
        tooltip = [
            alt.Tooltip('자치구:N'),
            alt.Tooltip('CCTV_비율:Q', format='.1%'),
            alt.Tooltip('CCTV_규모_{0}'.format(year)),
            alt.Tooltip('인구_규모_{0}'.format(year)),
        ],
        size = alt.Size('CCTV_비율:Q', scale=alt.Scale(range=[10,1000]), legend=None),
        order = alt.Order('CCTV_비율:Q', sort='descending')
        ).properties(
        width=600,
        height=500)

    reg = bb.transform_regression("인구_규모_{0}".format(year), "CCTV_규모_{0}".format(year)
                                   ).mark_line(opacity=0.5) 

    return alt.layer(
        bb,
        reg
    )  

### 3.2 2015년도 차트 시각화

In [ ]:
# 2015년도 차트를 파란색으로 시각화
bb_reg_2015 = bubble(2015, "blue")
bb_reg_2015

alt.LayerChart(...)

### 3.3 2020년도 차트 시각화

In [ ]:
# 2020년도 차트를 빨간색으로 시각화
bb_reg_2020 = bubble(2020,"red")
bb_reg_2020

alt.LayerChart(...)

### 3.4 이동궤적 시각화

In [ ]:
# 버블 이동궤적 시각화

move = alt.Chart(df).mark_line().transform_calculate(
    증가율= '(datum.CCTV_규모_2020 - datum.CCTV_규모_2015)/datum.CCTV_규모_2015'
).encode(
    x= alt.X('인구_규모_2015:Q', scale = alt.Scale(domain=[100000, 700000])),
    x2= alt.X2('인구_규모_2020:Q'),
    y = alt.Y('CCTV_규모_2015:Q', scale = alt.Scale(domain = [0,7000])),
    y2 = alt.Y2('CCTV_규모_2020:Q'),
    size = alt.Size('증가율:Q',legend = None),
    color = alt.Color('증가율:Q', legend = None),
    tooltip = [
        alt.Tooltip('자치구:N'),
        alt.Tooltip('증가율:Q', format='.1%')
    ]
).properties(
    width = 600,
    height = 400
)

move

alt.Chart(...)

### 3.5 시각화 결과 통합

In [ ]:
# 시각화 결과 통합

layer = alt.layer(
        bb_reg_2015,
        bb_reg_2020,
        move
).resolve_scale(
    color='independent',
    size = 'independent'
).properties(
    title = {
        "text": ["자치구별 인구 대비 CCTV 설치 규모", "(2015년 대 2020년)"],
        "subtitle": ["- 2015년은 푸른색, 2020년은 빨간색 원으로 표시", "- 원 크기는 인구 대비 CCTV 비율", "- 직선 두께와 색상은 2015년 대비 2020년 증가율"]
    }
)
layer

alt.LayerChart(...)

## 4. 단계구분도 시각화

### 4.1 자치구 위치 시각화

In [ ]:
points = alt.Chart(offices).mark_circle().encode(
    longitude = '경도:Q',
    latitude = '위도:Q',
    size = alt.value(50),
    tooltip = '자치구청'
).properties(
    width = 400, height = 300
)

points

alt.Chart(...)

### 4.2 단계구분도를 반환하는 함수 정의

In [ ]:
def map(rate, pop, cctv, year):
  ch = alt.Chart(seoul).mark_geoshape(
      stroke = '#aaa', strokeWidth=0.25
  ).transform_lookup(
      lookup = 'no',
      from_ = alt.LookupData(data=df, key='no', fields=['자치구', 'CCTV_규모_2015', '인구_규모_2015'])
  ).transform_calculate(
       CCTV_비율 = 'datum.CCTV_규모_2015 / datum.인구_규모_2015 *100'.format(year)
  ).encode(
    color=alt.Color('CCTV_비율:Q',legend=None),
    tooltip=['자치구:N',
             alt.Tooltip('CCTV_비율:Q', format='.2%'),
             alt.Tooltip('CCTV_규모_{0}'.format(year, ',d')),
            alt.Tooltip('인구_규모_{0}'.format(year, ',d'))]
).properties(
      title = {
          "text":"CCTV 설치 비율",
          "subtitle":"{0}년".format(year)
      },
      width = 400, height = 300
  )

  return ch

### 4.3 단계구분도 종합

In [ ]:
map2015 = alt.Chart(seoul).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup = 'properties.ESRI_PK',
    from_ = alt.LookupData(data=df, key='no', fields=['자치구', 'CCTV_규모_2015', '인구_규모_2015'])
).transform_calculate(
    cctv_rate=f'(datum.CCTV_규모_2015/datum.인구_규모_2015)*100'
).encode(
    color=alt.Color('cctv_rate:Q',legend=None),
    tooltip=['자치구:N',
             alt.Tooltip('cctv_rate:Q', format='.2%'),
             alt.Tooltip('CCTV_규모_2015:Q', format=',d'),
             alt.Tooltip('인구_규모_2015:Q', format=',d')]
).properties(
    title = {
        "text":"CCTV 설치 비율",
        "subtitle":"2015년"
    },
    width = 400, height = 300
)

m2015 = map2015 + points

map2020 = alt.Chart(seoul).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup = 'properties.ESRI_PK',
    from_ = alt.LookupData(data=df, key='no', fields=['자치구', 'CCTV_규모_2020', '인구_규모_2020'])
).transform_calculate(
    cctv_rate=f'(datum.CCTV_규모_2020/datum.인구_규모_2020)*100'
).encode(
    color=alt.Color('cctv_rate:Q',legend=None),
    tooltip=['자치구:N',
             alt.Tooltip('cctv_rate:Q', format='.2%'),
             alt.Tooltip('CCTV_규모_2020:Q', format=',d'),
             alt.Tooltip('인구_규모_2020:Q', format=',d')]
).properties(
    title = {
        "text":"CCTV 설치 비율",
        "subtitle":"2020년"
    },
    width = 400, height = 300
)

m2020 = map2020+points

alt.hconcat(m2015, m2020).configure_view(stroke=None)

alt.HConcatChart(...)